## Sparse Matrices and Graphs

In [2]:
# importing modules

import scipy.sparse as sp
import scipy.sparse.linalg
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

### Scipy.coo_matrix:

for example:

$$A =  \begin{bmatrix}0 & 1 & 0 & 0 \\0 & 0& 0& 2 \\0 & 0 & 3 & 0 \\4 & 0 & 0 & 0 \end{bmatrix} $$